In [1]:
import csv
import os
import matplotlib.pyplot as plt
import cv2
import numpy as np
import pandas
from joblib import dump
# from numba import jit
from joblib import dump, load
from sklearn.neural_network import MLPClassifier

In [2]:
def buildKernels(ksize):
    kernels = []
    for i in range(ksize * ksize):
        x = np.zeros((ksize, ksize))
        x[i % ksize][(ksize - 1) - i % ksize] = 1
        kernels.append(x)
    return kernels


# @jit(parallel=True)
def buildIMGS(kernels, img):
    images = []
    for kernel in kernels:
        cv_filter = cv2.filter2D(img, -1, kernel)
        images.append(cv_filter)
    return np.array(images)

def preprocess(Image: str, csv, ksize) -> None:
    print("Processing Started")
    frame = pandas.read_csv(csv)
    kernels = buildKernels(ksize)
    for index, row in frame.iterrows():
        path = os.path.join(Image, row["filename"])
        img = cv2.imread(path)

        # img = cv2.resize(img, (int(img.shape[1]*img_scale), int(img.shape[0]*img_scale)))

        imgs = buildIMGS(kernels, img)
        xmin, xmax, ymin, ymax = row["xmin"], row["xmax"], row["ymin"], row["ymax"]

        # xmin = xmin*img_scale
        # xmin = xmax*img_scale
        # xmax = int((xmax-xmin)*img_scale + xmin)
        # ymax = int((ymax-ymin)*img_scale + ymin)

        img_ = imgs[:, ymin:ymax, xmin:xmax, :]
        img_ = img_.reshape(img_.shape[0] * img_.shape[1] * img_.shape[2], 3)
        print("preprocessed: ", row["filename"])
        yield [img_, row["class"]]

In [3]:
Image = r"C:\Users\Emily\Documents\Bachelor_Drohnen_Bilder\PNG"
csv_ = r"C:\Users\Emily\Documents\Bachelor_Artefakte\image_data.csv"
ksize = 5

In [20]:
#clf = MLPClassifier(solver='adam', alpha=1e-5, hidden_layer_sizes=(5,3), random_state=1)
clf = load(r"C:\Users\Emily\Documents\Bachelor_Artefakte\modelle\filename2.joblib")

c:\users\emily\documents\notebooks\lib\site-packages\sklearn\base.py:310: UserWarning: Trying to unpickle estimator LabelBinarizer from version 0.22.2.post1 when using version 0.24.2. This might lead to breaking code or invalid results. Use at your own risk.
  warnings.warn(
c:\users\emily\documents\notebooks\lib\site-packages\sklearn\base.py:310: UserWarning: Trying to unpickle estimator MLPClassifier from version 0.22.2.post1 when using version 0.24.2. This might lead to breaking code or invalid results. Use at your own risk.
  warnings.warn(


In [10]:
gen = preprocess(Image, csv_, ksize)

In [21]:
a = next(gen)

preprocessed:  DJI_0002.png


In [22]:
X = a[0]

In [23]:
Y = np.chararray(len(X), itemsize=10, unicode=True)
Y[:] = a[1]

In [24]:
clf.score(X,Y)

0.0